In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos > 30]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.80,4.8,1,0.00,0.8,5,0.60,3.2,1,31
7,36889,ESPORTSTMNT02_2557053,19,1,2,5,1,20,21,4,...,0.00,0.0,1,0.00,0.4,0,0.00,0.0,0,32
33,36926,ESPORTSTMNT05_2530903,19,21,20,22,23,20,21,25,...,0.00,0.0,4,0.25,2.8,0,0.00,0.0,1,34
35,36928,ESPORTSTMNT05_2530937,0,20,21,1,22,3,24,2,...,0.33,6.8,2,0.00,1.8,1,0.00,8.0,0,35
37,36930,ESPORTSTMNT05_2530947,20,2,0,20,2,1,4,3,...,0.00,3.0,2,1.00,3.0,0,0.00,0.0,1,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,42737,ESPORTSTMNT03_2885433,0,21,22,1,5,2,4,20,...,0.58,4.6,0,0.00,0.0,24,0.67,3.1,1,31
3964,41155,ESPORTSTMNT03_2788956,20,2,20,0,1,4,5,21,...,0.92,16.9,20,0.55,3.7,35,0.40,2.6,0,36
3968,41159,ESPORTSTMNT03_2789756,20,21,20,24,21,1,5,23,...,0.25,1.8,42,0.40,3.3,2,0.00,2.6,0,37
3996,40957,ESPORTSTMNT03_2785705,1,19,21,20,2,23,5,1,...,0.00,1.4,1,0.00,1.5,9,0.44,3.2,0,38


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,qtd_eventos
2,19,2,20,24,2,4,20,3,6,1,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,31
7,19,1,2,5,1,20,21,4,1,3,...,0,0.00,0.0,1,0.00,0.4,0,0.00,0.0,32
33,19,21,20,22,23,20,21,25,5,24,...,0,0.00,0.0,4,0.25,2.8,0,0.00,0.0,34
35,0,20,21,1,22,3,24,2,4,7,...,3,0.33,6.8,2,0.00,1.8,1,0.00,8.0,35
37,20,2,0,20,2,1,4,3,22,5,...,1,0.00,3.0,2,1.00,3.0,0,0.00,0.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,0,21,22,1,5,2,4,20,24,3,...,19,0.58,4.6,0,0.00,0.0,24,0.67,3.1,31
3964,20,2,20,0,1,4,5,21,22,7,...,12,0.92,16.9,20,0.55,3.7,35,0.40,2.6,36
3968,20,21,20,24,21,1,5,23,22,20,...,8,0.25,1.8,42,0.40,3.3,2,0.00,2.6,37
3996,1,19,21,20,2,23,5,1,24,3,...,2,0.00,1.4,1,0.00,1.5,9,0.44,3.2,38


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained


c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Random Forest trained
Gradient Boosting Classifier trained


In [6]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [7]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.783333,0.846154,0.733333,0.785714,0.783333
1,Support Vector Machine (Linear Kernel),0.712500,0.791667,0.633333,0.703704,0.712500
2,Support Vector Machine (RBF Kernel),0.816667,0.857143,0.800000,0.827586,0.816667
3,Decission Tree,0.841667,0.920000,0.766667,0.836364,0.841667
4,Adaboost,0.833333,0.862069,0.833333,0.847458,0.833333
5,Random Forest,0.858333,0.923077,0.800000,0.857143,0.858333
6,Gradient Boosting Classifier,0.858333,0.923077,0.800000,0.857143,0.858333
